In [424]:
from IPython.core.debugger import set_trace
from collections import defaultdict
from math import *

## Part 1

In [444]:
def printCanvas(canvas):
    print('\n'.join([''.join(row) for row in canvas]))

def emptyCanvas(size):
    return [['*' for _ in range(size)] for _ in range(size)]
    
def rotate(mat, orientation):
    out = [['' for _ in range(len(mat[0]))] for _ in range(len(mat))]
    
    corners = [
        (0, 0),
        (0, len(mat[0])-1),
        (len(mat)-1, len(mat[-1])-1),
        (len(mat)-1, 0)
    ]
    
    nextCols = [
        (0, 1),
        (1, 0),
        (0, -1),
        (-1, 0)
    ]
    
    nextRows = [
        (1, 0),
        (0, -1),
        (-1, 0),
        (0, 1)
    ]
    
    start = corners[orientation // 90]
    nextCol = nextCols[orientation // 90]
    nextRow = nextRows[orientation // 90]
    i, j = start
    
    for row in range(len(mat)):
        for col in range(len(mat[row])):
            out[i][j] = mat[row][col]
            i += nextCol[0]
            j += nextCol[1]
            
        i = i + nextRow[0] if nextRow[0] else start[0]
        j = j + nextRow[1] if nextRow[1] else start[1]
        
    return out

def flipX(mat):
    return [row[::-1] for row in mat]

In [464]:
class Tile:
    """
    Borders: list of border strings, from left to right, in order of N, E, S, W
    """
    def __init__(self, id, borders, mat):
        self.id = id
        self.borders = borders
        self.mat = mat
        self.reset()
        
    def reset(self):
        self.used = False
        self.orientation = 0
        self.flipX = False
        self.neighbors = [-1] * 4
        
    def size(self, withBorder):
        return len(self.borders[0]) if withBorder else len(self.borders[0]) - 2
    
    def draw(self, canvas, top, left, withBorder):
        size = self.size(withBorder)
        if not withBorder:
            drawMat = [row[1:-1] for row in self.mat[1:-1]]
        else:
            drawMat = self.mat
        
        drawMat = rotate(drawMat, self.orientation)
        if self.flipX:
            drawMat = flipX(drawMat)
        
        i = 0
        j = 0
        for row in range(top, top+size):
            for col in range(left, left+size):
                if row >= len(canvas) or col >= len(canvas[row]) or i >= len(drawMat) or j >= len(drawMat[i]):
                    print(row, col, len(canvas), len(canvas[row]), top, left, i, j, len(drawMat), len(drawMat[i]))
                canvas[row][col] = drawMat[i][j]
                j += 1
            j = 0
            i += 1

In [571]:
lookup = defaultdict(list)
tiles = {}

tileId = -1
lineCount = 0
n = ''
e = ''
s = ''
w = ''
mat = []
with open('input.txt') as f:
    while True:
        line = f.readline()
        if not line:
            break
        
        if line[:4] == 'Tile':
            tileId = int(line.split(' ')[1][:-2])
            lineCount = 0
            mat = []
        elif line == '\n':
            tile = Tile(tileId, [n, e, s, w], mat)
            tiles[tileId] = tile
            for border in tile.borders:
                lookup[border].append(tile)
            n = e = s = w = ''
        else:
            line = line.strip()
            mat.append([c for c in line])
            if lineCount == 0:
                n = line
            elif lineCount == 9:
                s = line[::-1]
            e += line[-1]
            w = line[0] + w
            lineCount += 1

In [572]:
neighbors = {}
corners = []
product = 1
for id, tile in tiles.items():
    matches = []
    for b in tile.borders:
        for t in lookup[b] + lookup[b[::-1]]:
            if t.id != tile.id:
                matches.append(t)
    neighbors[id] = [t.id for t in matches]
    if len(matches) == 2:
        product *= tile.id
        corners.append(tile.id)
        print(tile.id)
#     print(f'{tile.id}: {len(matches)}')

print()
print(product)

1327
3571
3391
1823

29293767579581


## Part 2

t0|rev|t1
-|-|-
T|T|T
T|F|F
F|T|F
F|F|T

In [573]:
def solve(tid):
    tile = tiles[tid]
    tile.used = True
    for i, b in enumerate(tile.borders):
        for t in lookup[b] + lookup[b[::-1]]:
            if t.id != tile.id:
                i2, j2 = orient(tile, t, i, b, not t.used)
                if not t.used:
                    solve(t.id)

dirs = [
    (-1, 0),
    (0, 1),
    (1, 0),
    (0, -1)
]
def flooddraw(tid, canvas, top, left, withBorder, drawn):
    drawn.add(tid)
    tile = tiles[tid]
    tile.draw(canvas, top, left, withBorder)
    size = tile.size(withBorder)
    for i, nid in enumerate(tile.neighbors):
        if nid != -1 and nid not in drawn:
            d = dirs[i]
            nextTop = top + size * d[0]
            nextLeft = left + size * d[1]
            flooddraw(nid, canvas, nextTop, nextLeft, withBorder, drawn)

def orient(t0, t1, i, b, setOrient):
    i2 = (i + (t0.orientation // 90)) * (-1 if t0.flipX else 1)
    i2 = ((i2 % 4) + 4) % 4
    j2 = (i2 + 2) % 4
    t0.neighbors[i2] = t1.id
    t1.neighbors[j2] = t0.id
    
    if setOrient:
        for j, c in enumerate(t1.borders):
            if c == b:
                rev = False
                break
            elif c == b[::-1]:
                rev = True
                break

        t1.flipX = t0.flipX == rev
        t1.orientation = ((-1 if t1.flipX else 1) * j2 - j)
        t1.orientation = (((t1.orientation % 4) + 4) % 4) * 90
    
    return i2, j2

In [574]:
solve(corners[0])

In [575]:
for tid in corners:
    if tiles[tid].neighbors[0] == -1 and tiles[tid].neighbors[3] == -1:
        start = tid
        break

withBorder = False
size = int(sqrt(len(tiles))) * tiles[start].size(withBorder)
canvas = emptyCanvas(size)
flooddraw(start, canvas, 0, 0, withBorder, set())

In [576]:
with open('output.txt', 'w') as f:
    for row in canvas:
        f.write(''.join(row) + '\n')

In [577]:
printCanvas(canvas)

#..##.#.##....##.#.....#..#......##...#.....##.....#.#...#....##......##.##..........#.#........
...........#...#...........#..##....#....#.....#.......##...##.....#.............#..##....#...#.
#........#.#.............#..#..#........##....##.....#..#.#...#..#.##.##....#.....#.#...........
..#......#.#....#.#.#....#.#...##...#..##..##..#.#...#.###...#.#...#..#.#..#.#.#..#.##.###.#....
..#.#.....#.....##..#.#.........###.#.##.............#.##....##.#....#..#.....#..###............
.###.........#.......#.#..###..#.#..##.#..#.#.......##...##.#.#.....#.............##....#.#.....
..##.......#............#...#.##...#..###.......##..#...#.#....#..#.#.......#...#..#............
...#.....#..##.###..#...##.#####..#...##.#.#.#.....#...........#...#......#.#....##..#.#..#...#.
..#..#..#..#.....#...........#.##..#........##.###.#....##..###......#..#......###...........#..
....#.##.###..#..#.##.......#...#.....#.#....#.........##.#....#.##....#......#....##..#.#...##.
...#..#..##.#......#..#.#....#

### Find the monster

In [578]:
kernel = []
with open('kernel.txt') as f:
    while True:
        line = f.readline()
        if not line:
            break
            
        kernel.append([c for c in line[:-1]])

In [579]:
printCanvas(kernel)

                  # 
#    ##    ##    ###
 #  #  #  #  #  #   


In [580]:
def matchKernel(canvas, kernel, top, left):
    for i in range(len(kernel)):
        for j in range(len(kernel[0])):
            if kernel[i][j] == '#' and canvas[top+i][left+j] != '#':
#                 print(i, j, canvas[top+i][left+j])
                return False
    return True

def findSeaMonsters(canvas, kernel):
    for o in [0, 90, 180, 270]:
        for f in [False, True]:
            mat = rotate(canvas, o)
            if f:
                mat = flipX(mat)

            seaMonsters = []
            for i in range(len(mat) - len(kernel) + 1):
                for j in range(len(mat[0]) - len(kernel[0]) + 1):
                    if matchKernel(mat, kernel, i, j):
                        seaMonsters.append((i, j))

            if seaMonsters:
                return seaMonsters, o, f

In [581]:
seaMonsters, o, f = findSeaMonsters(canvas, kernel)
print(f'Found {len(seaMonsters)} when o={o}, f={f}:\n{seaMonsters}')

Found 39 when o=90, f=False:
[(1, 4), (1, 26), (1, 72), (5, 48), (9, 67), (10, 3), (10, 28), (15, 30), (17, 3), (19, 53), (22, 26), (23, 3), (26, 46), (30, 8), (30, 70), (32, 40), (37, 4), (37, 27), (37, 50), (38, 73), (43, 49), (45, 3), (49, 48), (50, 26), (54, 4), (55, 74), (56, 51), (61, 2), (65, 67), (66, 23), (71, 5), (72, 43), (76, 12), (77, 47), (81, 73), (82, 32), (85, 2), (87, 54), (88, 26)]


In [582]:
def countHash(mat):
    count = 0
    for row in mat:
        for col in row:
            if col == '#':
                count += 1
    return count

In [583]:
countHash(canvas) - countHash(kernel) * len(seaMonsters)

1989